<a href="https://colab.research.google.com/github/lourencocavalcante/Neurocomputacao/blob/main/INPE_CAP351_Projeto02_MLP_Lourenco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introdução**

Aprendizado Não Supervisionado são modelos de Machine Learnig que possuem poucos ou nenhum dado histórico para se basear e conseguir prever os resultados, eles não necessitam de um conjunto de dados que digam quais são as variáveis de saídas corretas, para modelar um algoritmo preditivo. Esses modelos conseguem criar estruturas de dados com base em relações entre as variáveis ou detectar algumas tendências.

No presente notebook contém diversos experimentos e testes com a rede Multi-layer Perceptron (MLP). Foi selecionado um datasets (não trivial) no qual foi divido em treino, validação e teste, e utilizado para treino de modelos MLP em dois problemas: classificação e regressão. Os Dados são de um equipamento chamado **Disdrômetro RD80** (também chamado de **Joss**). Instalado próximo à torre **ATTO**, em um sítio chamado Campina, na região Amazônica. O disdrômetro RD80 mede a distribuição das gotas de chuva que chegam à superfície.

Foram criadas 5 topologias diferentes, variando o número de camadas. O algoritmo utilizado foi SGD, e foram aplicadas diferentes hyperparâmetros afim de avaliar o impacto do uso do Momentum e da Regularização (L2).

Este notebook pode ser acessado no **github**: https://github.com/lourencocavalcante/Neurocomputacao.git

# **Bibliotecas/Pacotes**

In [2]:
!pip install minisom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for minisom: filename=MiniSom-2.3.0-py3-none-any.whl size=9018 sha256=c1e6629687eba3728230158a6e8c6dbb77fa537d0bba48cd628bbff60050a703
  Stored in directory: /root/.cache/pip/wheels/d4/ca/4a/488772b0399fec45ff53132ed14c948dec4b30deee3a532f80
Successfully built minisom


In [3]:
from minisom import MiniSom

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_blobs
from sklearn.manifold import TSNE

from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage

import datetime
import time

from google.colab import drive
drive.mount('/content/drive')

#print(tf.__version__)

Mounted at /content/drive


# **Dataset**
 
Os Dados são de um equipamento chamado **Disdrômetro RD80** (também chamado de **Joss**). Instalado próximo à torre **ATTO**, em um sítio chamado Campina, na região amazônica, o Disdrômetro RD80 mede a distribuição das gotas de chuva que chegam à superfície.

## **Importação, Formatação, Tratamento e Visualização dos dados**

In [6]:
#Importando dos dados
df_rd80_dataset = pd.read_csv("/content/drive/MyDrive/MestradoCAP/Lourenco/cap351/Datasets/Proj2/df_class_rd80.csv")
df_rd80_dataset = pd.DataFrame(df_rd80_dataset)

#Aqui verificamos o tamanho e quantidade de atributos do Dataset
print('Tamanho do dataset: ',df_rd80_dataset.shape[0], ', quantidade de atributos: ', df_rd80_dataset.shape[1])

Tamanho do dataset:  315420 , quantidade de atributos:  13


In [7]:
print(df_rd80_dataset.dtypes)

Datetime                         object
Rain Intensity (mm/h)           float64
radar reflectivity (1/mm6m3)    float64
Liquid watercontent (g/m3)      float64
Mean weight diameter(mm)        float64
Time integration (s)            float64
Light_Rain                        int64
Moderate_Rain                     int64
Heavy_Rain                        int64
Very_Heavy_Rain                   int64
Without_Rain                      int64
Class_Rain_id                     int64
Class_Rain                       object
dtype: object


In [8]:
df_rd80_dataset = df_rd80_dataset.set_index('Datetime')

In [9]:
df_rd80_dataset.isna().sum()

Rain Intensity (mm/h)           0
radar reflectivity (1/mm6m3)    0
Liquid watercontent (g/m3)      0
Mean weight diameter(mm)        0
Time integration (s)            0
Light_Rain                      0
Moderate_Rain                   0
Heavy_Rain                      0
Very_Heavy_Rain                 0
Without_Rain                    0
Class_Rain_id                   0
Class_Rain                      0
dtype: int64